In [1]:
import os
import glob
import pandas as pd
os.chdir("/Users/jisha/Desktop/Lambda_Course_Work/Coursework/Work/3. Labs/Workspace_Datasets/Datasets/Crime_Data/csv")

In [2]:
path = os.getcwd()

In [3]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [4]:
# combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
# export to csv
combined_csv.to_csv("../combined_csv.csv", index=False)

In [5]:
import pandas as pd
import numpy as np

In [6]:
crime_filepath = os.path.join(path, "../combined_csv.csv")
combined_csv = pd.read_csv(crime_filepath)
print(combined_csv.shape)
combined_csv.head()

(8105, 14)


,Unnamed: 0,City,State,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson
0,4,Acworth,GA,22921,16,0,2,1,13,437,24,399,14,0
1,5,Adairsville,GA,4953,7,0,2,0,5,83,15,59,9,0
2,6,Albany,GA,74989,790,12,32,165,581,3452,729,2489,234,17
3,7,Alma,GA,3438,10,0,0,2,8,111,36,69,6,0
4,8,Alpharetta,GA,67411,75,1,7,11,56,991,97,849,45,0


## Clean Data

### Remove Cities with a 0 populations

In [7]:
# Population cannot be 0
combined_csv.loc[combined_csv['Population']==0.0]

,Unnamed: 0,City,State,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson
693,7,Baltimore City,MD,0,0,0,0,0,0,0,0,0,0,0
7055,77,Sunriver,OR,0,1,0,0,0,1,54,9,43,2,0


In [8]:
combined_csv = combined_csv[combined_csv.Population != 0]

### Fill empty columns with violent crime and property crime sums if available.
- property crime is the sum of 'Burglary', 'Larceny- theft', 'Motor vehicle theft'
- violent crime is the sum of 'Murder and nonnegligent manslaughter', 'Rape','Robbery', 'Aggravated assault'


In [9]:
violent_crimes = ['Murder and nonnegligent manslaughter', 'Rape','Robbery', 
                  'Aggravated assault']

In [10]:
combined_csv['Violent crime']= combined_csv[violent_crimes].sum(axis=1)

In [11]:
property_crimes = ['Burglary', 'Larceny- theft', 'Motor vehicle theft']

In [12]:
combined_csv['Property crime']= combined_csv[property_crimes].sum(axis=1)

## Feature Engineering
- Creating a crime rate per 1000 column 
- Creating a crime rating column


### Crime Rate
Crime Rate per 1,000 inhabitants: This represents the number of Index offenses per 1,000 inhabitants.For example: What is the crime rate for a municipality with 513 Index offenses (murder, rape, robbery,aggravated assault, burglary, larceny-theft and motor vehicle theft), with a population of 8,280?
513 (Index offenses) ÷ 8,280 (population) = .061957 x 1,000 = 62.0 (crime per 1,000 inhabitants) 

https://www.njsp.org/info/ucr2000/pdf/calc_ucr2000.pdf

In [13]:
# Crime is broken down into violent crime and property crime
# Arson is reported in conjunction with property or violent crime depending on the nature
columns = ['Violent crime', 'Property crime']

In [14]:
combined_csv[columns] = combined_csv[columns].astype(int)

In [15]:
def sum_frame_by_column(frame, new_col_name, list_of_cols_to_sum, divideby):
    frame[new_col_name] = ((frame[list_of_cols_to_sum].astype(int).sum(axis=1)/frame[divideby].astype(int))*1000).round(2)
    return(frame)

In [16]:
sum_frame_by_column(combined_csv, 'Crime Rate (per 1000 residents)', columns, 'Population' )

,Unnamed: 0,City,State,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson,Crime Rate (per 1000 residents)
0,4,Acworth,GA,22921,16,0,2,1,13,437,24,399,14,0,19.76
1,5,Adairsville,GA,4953,7,0,2,0,5,83,15,59,9,0,18.17
2,6,Albany,GA,74989,790,12,32,165,581,3452,729,2489,234,17,56.57
3,7,Alma,GA,3438,10,0,0,2,8,111,36,69,6,0,35.19
4,8,Alpharetta,GA,67411,75,1,7,11,56,991,97,849,45,0,15.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8100,249,West Plains,MO,12284,52,0,10,2,40,709,126,496,87,0,61.95
8101,250,Wheaton,MO,693,0,0,0,0,0,0,0,0,0,0,0.00
8102,251,Willard,MO,5616,7,0,1,0,6,74,14,55,5,0,14.42
8103,252,Willow Springs,MO,2103,13,0,4,0,9,71,22,41,8,1,39.94


In [17]:
min(combined_csv['Crime Rate (per 1000 residents)']), max(combined_csv['Crime Rate (per 1000 residents)'])

(0.0, 25625.0)

In [18]:
# Just wanted to see the type of crime for such a high percentage
combined_csv.loc[combined_csv['Crime Rate (per 1000 residents)']==25625.0]

,Unnamed: 0,City,State,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson,Crime Rate (per 1000 residents)
4098,71,Lakeside,CO,8,0,0,0,0,0,205,1,202,2,0,25625.0


### Using pd.cut() to create a Crime Ratings Columns

In [19]:
combined_csv['Crime Rate (per 1000 residents)'].describe()

count     8103.000000
mean        25.214522
std        298.766689
min          0.000000
25%          7.945000
50%         15.440000
75%         26.920000
max      25625.000000
Name: Crime Rate (per 1000 residents), dtype: float64

In [20]:
combined_csv['Crime Rating'] = pd.qcut(combined_csv['Crime Rate (per 1000 residents)'], q=3, labels=['Low', 'Medium', 'High'])

In [21]:
combined_csv.head()

,Unnamed: 0,City,State,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson,Crime Rate (per 1000 residents),Crime Rating
0,4,Acworth,GA,22921,16,0,2,1,13,437,24,399,14,0,19.76,Medium
1,5,Adairsville,GA,4953,7,0,2,0,5,83,15,59,9,0,18.17,Medium
2,6,Albany,GA,74989,790,12,32,165,581,3452,729,2489,234,17,56.57,High
3,7,Alma,GA,3438,10,0,0,2,8,111,36,69,6,0,35.19,High
4,8,Alpharetta,GA,67411,75,1,7,11,56,991,97,849,45,0,15.81,Medium


In [22]:
combined_csv.to_csv("../fbi_crime_uscities.csv", index=False)

In [23]:
fbi_crime_filepath = os.path.join(path, "../fbi_crime_uscities.csv")
fbi_crime_uscities = pd.read_csv(fbi_crime_filepath)
fbi_crime_uscities.head()

,Unnamed: 0,City,State,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson,Crime Rate (per 1000 residents),Crime Rating
0,4,Acworth,GA,22921,16,0,2,1,13,437,24,399,14,0,19.76,Medium
1,5,Adairsville,GA,4953,7,0,2,0,5,83,15,59,9,0,18.17,Medium
2,6,Albany,GA,74989,790,12,32,165,581,3452,729,2489,234,17,56.57,High
3,7,Alma,GA,3438,10,0,0,2,8,111,36,69,6,0,35.19,High
4,8,Alpharetta,GA,67411,75,1,7,11,56,991,97,849,45,0,15.81,Medium


In [24]:
fbi_crime_uscities.loc[fbi_crime_uscities['Crime Rate (per 1000 residents)']>=100]

,Unnamed: 0,City,State,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson,Crime Rate (per 1000 residents),Crime Rating
253,148,Lake Delton,WI,3005,24,0,4,3,17,458,23,417,18,0,160.40,High
421,316,West Milwaukee,WI,4107,30,1,0,11,18,1467,31,1411,25,0,364.50,High
743,58,Ocean City,MD,6905,91,0,5,18,68,720,94,615,11,0,117.45,High
1062,30,Balcones Heights,TX,3366,38,1,0,8,29,304,23,244,37,0,101.60,High
1261,229,Humble,TX,16157,189,1,34,58,96,1681,110,1394,177,1,115.74,High
1493,461,South Padre Island,TX,2805,44,0,20,6,18,365,18,318,29,0,145.81,High
1505,473,Sunset Valley,TX,683,5,0,0,2,3,135,4,126,5,0,204.98,High
1718,152,Mangonia Park,FL,2033,33,0,3,11,19,177,10,160,7,1,103.30,High
1723,157,Medley,FL,896,11,0,0,3,8,179,28,130,21,0,212.05,High
2091,236,Mackinac Island,MI,468,2,0,0,0,2,94,0,93,1,0,205.13,High


In [25]:
len(fbi_crime_uscities.loc[fbi_crime_uscities['Crime Rate (per 1000 residents)']>=100])

48